# 仅作数据分析代码记录
# 分析时间：20220501
# 记录时间：20220501
# 项目：ASV相对丰度两两组间求log2FC

In [ ]:
### 导入模块部分
import pandas as pd

### 读文件
df1 = pd.read_excel(r'D:\Work\分析项目\wxy\宏基因组与16S丰度计算\ASV丰度(加测数据)\ASV_relative_depoint.xlsx'
                    , header=0, index_col=0)
df1.drop(["Taxonomy"], axis=1, inplace=True)
df1 = df1.T

"""要处理的数据存在df1里"""

### 把数据分组 分组信息中的组名转换为列表
dfGroup = df1.groupby('GroupByTimeDiet')  # 按'GroupByTimeDiet'分组
KeyList = list(dfGroup.groups.keys())  # 分组组名转为列表
len(KeyList)

# 转换数据类型
for i in df1.columns:
    if i != "GroupByTimeDiet":
        df1[i] = df1[i].astype(float, errors='raise')

# 1.找到相对丰度表中最小的值
"""失败 0值小数点后的位数不一样，没法替换成空值。这样求出来的最小值只能是0
"""
# 2.相对丰度表里所有的值都加上这个最小的值（处理0的问题）第一步失败，改为每个值都加0.0001

### 删除包含字符串的列
df_de = df1.drop(["GroupByTimeDiet"], axis=1, inplace=False)

### df_de中每个元素加0.0001
df_de = df_de.applymap(lambda x: x + 0.0001)

### 把df1["GroupByTimeDiet"]插入df_de第一列
df_de.insert(0, "GroupByTimeDiet", df1["GroupByTimeDiet"])

### df1_AddNum = 加完0.0001的dataframe
df1_AddNum = df_de

# 3.求丰度表每个条目在每个组中的均值 比如ASV0 在ND0组的均值

### 对数据分组，按“GroupByTimeDiet”列分组
df1_AddNum.groupby("GroupByTimeDiet")

### 再获取每一列，计算组内均值
SeriesList = []  # 定义一个存series的列表
for i in df1_AddNum.columns:
    if i != "GroupByTimeDiet":
        SeriesList.append(df1_AddNum.groupby("GroupByTimeDiet")[i].mean())
    else:
        print("分组列，不计算")

# 4.结果存为新的df
pd.concat(SeriesList, axis=1).T

In [ ]:
dfSeriesConcat = pd.concat(SeriesList, axis=1).T
dfSeriesConcat

In [ ]:

# 5.用求出来的均值计算组间log2FC 比如log2(ND2的ASV0/ND0的ASV0)

import numpy as np

# dfSeriesConcat['ND2/ND0']  = dfSeriesConcat.apply(lambda f: log2fc(f['ND2'],f['ND0']), axis=1)
counter = len(dfSeriesConcat.columns)
i = 0

while i < counter:
    data1 = dfSeriesConcat[dfSeriesConcat.columns[i]]  # data1 = dfSeriesConcat的一列
    p = i + 1
    while p < counter:
        data2 = dfSeriesConcat[dfSeriesConcat.columns[p]]  # data2 = dfSeriesConcat的一列
        ColName = dfSeriesConcat.columns[p] + "/" + dfSeriesConcat.columns[i]
        dfSeriesConcat[ColName] = np.log2(data2 / data1)
        p += 1
    i += 1

dfSeriesConcat.to_excel(r'D:\Work\分析项目\wxy\宏基因组与16S丰度计算\ASV丰度(加测数据)\ASV_relative_depoint_log2FC.xlsx')
